In [1]:
import pandas as pd
import numpy as np

In [2]:
import os

FILENAME = "cleaned_content_based.csv"
DATA_PATH = "archive"
RECOMMENDER_TYPE = "content_based"
PROJECT_ROOT_DIR = "."
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", RECOMMENDER_TYPE)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, extension="png", resolution=300):  # Función para guardar las figuras que se vayan generando
    img_path = os.path.join(IMAGES_PATH, fig_id + "." + extension)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(img_path, format=extension, dpi=resolution)

In [3]:
from matplotlib import pyplot as plt

# Configuración de parámetros de matplotlib

plt.rc("font", size=14)
plt.rc("axes", labelsize=14, titlesize=14)
plt.rc("legend", fontsize=14)
plt.rc("xtick", labelsize=10)
plt.rc("ytick", labelsize=10)

In [4]:
df = pd.read_csv(os.path.join(DATA_PATH, FILENAME), low_memory=False)

# Content Based Filtering

El primer modelo que hemos creado ha sido un recomendador muy simple, basado en las películas y su popularidad, con un filtro del género de la película. Sin embargo, este sistema recomienda las mismas películas a todos los usuarios, por lo que si un usuario quiere recomendaciones sobre películas de Romance, siempre recibirá las mismas recomendaciones que otros usuarios que busquen ese mismo género.

En caso de que una persona quisiera conocer películas de un mismo director, actor o que tenga una trama similar a otra pero con diferentes géneros, el recomendador simple que hemos elaborado no serviría. Vamos a hacer dos tipos de recomendadores:

- Basado en el contenido de la descripción de la película
- Basado en los metadatos (director, actores, géneros...)

De esta forma desarrollaremos dos sistemas de recomendación superiores al que hicimos anteriormente.

## Basado en la descripción

Durante el procesamiento de los datos en _notebook_ EDA00, hicimos una característica que englobaba la reseña y el _tagline_ de las películas (en caso de haber _tagline_). A partir de esta descripción, podemos realizar un sistema de recomendación basado en la similaridad de las películas. Recordemos que en nuestro caso la característica que engloba todo se llama _**description**_.

In [5]:
df.head(3)

,genres,id,title,overview,description,popularity,vote_average,vote_count,cast,keywords,director,metadata
0,"['animation', 'comedy', 'family']",862,Toy Story,"Led by Woody, Andy's toys live happily in his ...","Led by Woody, Andy's toys live happily in his ...",21.946943,7.7,5415.0,"['tomhanks', 'timallen', 'donrickles']","['jealousi', 'toy', 'boy', 'friendship', 'frie...",['johnlasseter'],animation comedy family jealousi toy boy frien...
1,"['adventure', 'fantasy', 'family']",8844,Jumanji,When siblings Judy and Peter discover an encha...,When siblings Judy and Peter discover an encha...,17.015539,6.9,2413.0,"['robinwilliams', 'jonathanhyde', 'kirstendunst']","['boardgam', 'disappear', ""basedonchildren'sbo...",['joejohnston'],adventure fantasy family boardgam disappear ba...
2,"['romance', 'comedy']",15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,A family wedding reignites the ancient feud be...,11.712900,6.5,92.0,"['waltermatthau', 'jacklemmon', 'ann-margret']","['fish', 'bestfriend', 'duringcreditssting', '...",['howarddeutch'],romance comedy fish bestfriend duringcreditsst...


Lo primero que vamos a realizar es una tokenización del texto, eliminando los _stopwords_ y después le vamos a pasar un _stemmer_ tras tokenizar el texto.

In [6]:
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import nltk
import re

stop_w = set(stopwords.words("english"))  # Stopwords
remove_non_alpha = lambda x: re.sub(r"[^a-zA-Z]", " ", x)  # Elimina aquello que no sean palabras
tokenize = lambda x: nltk.tokenize.word_tokenize(x)  # Tokeniza cada entrada
stemmer = SnowballStemmer("english")
stemmer_fn = lambda desc: " ".join([stemmer.stem(w) for w in desc if w.lower() not in stop_w])  # Paso de estandarización

In [7]:
for fn in (remove_non_alpha, tokenize, stemmer_fn):
    df["description"] = df["description"].transform(func=fn)

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(analyzer="word", ngram_range=(1, 2), stop_words="english", min_df=0.)  # Redundancia de stopwords
tfidf_matrix = tfidf.fit_transform(df["description"])

In [9]:
tfidf_matrix.shape

(44467, 974653)

In [10]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [11]:
indices_mapper = pd.Series(df.index, index=df["title"]).drop_duplicates()

In [12]:
movie = "The Avengers"
idx = indices_mapper[movie]
print(idx)
scores = list(enumerate(cosine_sim[idx]))
scores = sorted(scores, key=lambda x: x[1], reverse=True)
scores = scores[0:11]
idx_ = [s[0] for s in scores]
df["title"].iloc[idx_]

title
The Avengers     2027
The Avengers    17708
dtype: int64


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()